In [ ]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
# from nltk import word_tokenize

In [ ]:
tweets = pd.read_csv("./Tweets.csv")

In [ ]:
tweets.head()


In [ ]:
tweets.info()

In [ ]:
tweets.shape

In [ ]:
#this
review_dataset["tweet_coord"].dropna()
# or this
review_dataset["tweet_coord"][~review_dataset["tweet_coord"].isnull()]


In [ ]:
tweets.text.tail()

### Word Cloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS
# Building a word cloud for negative tweet text
negative_tweets = tweets["text"][tweets["airline_sentiment"] =="negative"]
joined_negative_tweets = "".join(negative_tweets)

my_stop_words = set(STOPWORDS)
# reg = r"@"|
my_stop_words = my_stop_words.union(["@VirginAmerica", "united","flight","AmericanAir","USAirways","SouthwestAir","JetBlue"])
word_cloud = WordCloud(stopwords=my_stop_words, regexp=r'\b[^\d\W][^\d\W]+\b').generate(joined_negative_tweets)

#show word cloud
plt.imshow(word_cloud, interpolation="bilinear")
plt.axis("off")




In [ ]:
from wordcloud import WordCloud, STOPWORDS
# Building a word cloud for negative tweet text
# positive_neutral_tweets = tweets["text"][~tweets["airline_sentiment"] =="negative"]
positive_neutral_tweets = tweets["text"][tweets["airline_sentiment"] !="negative"]
joined_positive_neutral_tweets = "".join(positive_neutral_tweets)

my_stop_words = set(STOPWORDS)
# reg = r"@"|
my_stop_words = my_stop_words.union(["@VirginAmerica", "united","flight","AmericanAir","USAirways","SouthwestAir", "JetBlue","VirginAmerica","co", "t"])
word_cloud = WordCloud(background_color="white", stopwords=my_stop_words, regexp=r'\b[^\d\W][^\d\W]+\b').generate(joined_positive_neutral_tweets)

#show word cloud
plt.imshow(word_cloud, interpolation="bilinear")
plt.axis("off")




### Data Preprocessing and Feature Engineering

In [ ]:
tweets = tweets[["text","airline_sentiment"]]

In [ ]:
#Encode the target label
from sklearn.preprocessing  import LabelEncoder
le = LabelEncoder()
tweets.airline_sentiment = le.fit_transform(tweets.airline_sentiment)

In [ ]:
# Defie the text preprocessor function
def preprocessor(text):
    text = re.sub(r"@[d W]+", "", text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    
    return text
 

In [ ]:
# Cleaning the text
tweets.text = tweets.text.apply(preprocessor)

In [ ]:
# Add text_length feature
from nltk import word_tokenize
tokens = [word_tokenize(word) for word in tweets.text]
length =[] 
for i in range(len(tokens)):
    length.append(len(tokens[i]))
    
tweets["text_length"] = length

## Building the Classifier

In [ ]:
Form nltk import word_tokenize, PoterStemmer, stopwords

#defining the stop words
stop_words = stopwords
stop_words_2 = set(stopwords).union("AmericanAir","USAirways","SouthwestAir", "JetBlue","VirginAmerica")

def tokenizer(text):
    return [word_tokenize(text)]

def tokenizer_porter(text):
    return [poterStemmer().stem(text)]

In [ ]:
# Define X and y
y = tweets.airline_sentiment
X = tweets.drop('airline_sentiment', axis=1)


In [ ]:
from sklearn.model_selection import train_test_split
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=456)

#### Traing a Logistic Regression Model

In [ ]:
from  sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_moddel import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(lowercase=False)

param_grid =[
    {
        "vect__ngram_range" : [(1,1),(1,2)],
        "vect__stop_words" : [None, stop_words_1],
        "vect__tokenize" : [tokenizer, tokenizer_porter],
        "clf__penalty" : ["l2","l1"],
        "clf__C" : [0.1, 10.0]
    },
    {
        "vect__ngram_range" : [(1,1),(1,2)],
        "vect__stop_words" : [stop_words_1, stop_words_2],
        "vect__tokenize" : [tokenizer],
        "vect__use_idf" : [False],
        "vect__norm" : [None],
        "clf__penalty" : ["l2","l1"],
        "clf__C" : [1.0, 10.0]
        
    }
]

lr_tfidf = Pipeline([("vect", tfidf), ("clf", LogisticRegression())])

gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid, scoring='accuracy', cv=5, verbose=2, n_jobs=1)

gs_lr_tfidf.fit(X_train, y_train)

    



In [ ]:
print(gs_lr_tfidf.best_params_)
print(gs_lr_tfidf.best_score)

In [ ]:
y_predicted = gs_lr_tfidf.best_estimator_.predict(X_test)

In [ ]:
gs_lr_tfidf.best_estimator_.score(X_test, y_test)

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS

# # Build the vectorizer
# vect = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS, ngram_range=(1, 2), max_features=200,
#                     token_pattern=r'\b[^\d\W][^\d\W]+\b').fit(tweets.text)
# # Create sparse matrix from the vectorizer
# X_tfidf = vect.transform(tweets.text)

# # Create a DataFrame
# X_tfidf_transformed = pd.DataFrame(X_tfidf.toarray(), columns=vect.get_feature_names_out())
# X_tfidf_transformed.head()
# # can also do this: max_df= 0.9, min_df=0.01

In [ ]:
tweets

In [ ]:
data = pd.concat([ X_tfidf_transformed, tweets], axis=1)

In [ ]:
data

In [ ]:
# Lastly, build the classifier


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Train a logistic regression
log_reg = LogisticRegression(solver="newton-cg", C= 1).fit(X_train, y_train)

# Predict the labels
y_predicted = log_reg.predict(X_test)

# Print accuracy score and confusion matrix on test set
print('Accuracy on the test set: ', accuracy_score(y_test, y_predicted))
# print(confusion_matrix(y_test, y_predicted)/len(y_test))
print(confusion_matrix(y_test, y_predicted))